# Jetpack Compose - Konwersja - Podstawy

W tej aplikacji przyjrzymy się konwertowaniu `Flow` do `StateFlow` i `SharedFlow`.

<img src="https://media1.giphy.com/media/v1.Y2lkPTc5MGI3NjExdDZ1N3hvOXM2NXo1cTd3NDM2enlqOWlzOTNsYjJ1M2Uycm52NzlicCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/oVRpNx8YgOGJmka80w/giphy.gif" width="200" />

Możemy spotkać się z sytuacją w której chcemy przekonwertować `Flow` na `StateFlow`, lub `SharedFlow` aby zapewnić odpowednie zachowanie na cyklach życia. Możemy to zrobić za pomcą metod `stateIn` i `sharedIn`

Nasza aplikacja będzie prostym licznikiem, w którym po upłynięciu określonego czasu zmienimy wartość i zwiększymy stan licznika. Aplikacja będzie posiadała dwa ekrany. Chcemy zapewnić że po przejściu na drugi ekran, emisja zostanie wstrzymana, po powrocie wznowiona - jest to najczęściej pożądane zachowanie.

Rozpocznijmy od zaimplementowania `CounterViewModel`

In [ ]:
class CounterViewModel : ViewModel() {
    private var currentVal = 0

    val counter = flow {
        while (true){
            delay(500L)
            emit(currentVal++)
        }
    }.stateIn(
        viewModelScope,
        SharingStarted.WhileSubscribed(),
        0
    )
}

Używamy `Flow` do emitowania kolejnych wartości licznika. Strumień jest następnie przekształcony za pomocą `stateIn` w `StateFlow`, który jest dostępny dla subskrybentów.
- `private var currentVal = 0` - Prywatna zmienna, która przechowuje aktualną wartość licznika.
- `val counter = flow { ... }` - Deklaruje counter, który jest strumieniem `Flow`. Wewnątrz bloku `flow { ... }` emitowane są wartości.
- `emit(currentVal++)` - Emitowanie wartości do strumienia `Flow`.
- `.stateIn(viewModelScope, SharingStarted.WhileSubscribed(), 0)` - Przekształcenie strumienia `Flow` w `StateFlow`, który jest dostępny dla subskrybentów. Funkcja `stateIn` konwertuje strumień `Flow` na `StateFlow` z określonym zachowaniem.
    - `viewModelScope` - Określa zasięg, w którym `StateFlow` jest aktywny - dopóki `ViewModel` jest aktywny.
    - `SharingStarted.WhileSubscribed()` - Określa, kiedy wartość `StateFlow` jest udostępniana. W tym przypadku, jest udostępniany tylko wtedy, gdy jest subskrybowany przez obserwatora.

Zaimplementujmy ui, oraz dodajmy go jako subskrybenta

In [ ]:
sealed class Screens(val route: String) {
    object MainScreen : Screens("main_screen")
    object SecondScreen : Screens("second_screen")
}

In [ ]:
@Composable
fun MainScreen(onSecondScreen: () -> Unit) {

    val viewModel: CounterViewModel = viewModel()
    val counter = viewModel.counter.collectAsStateWithLifecycle()

    Column(
        Modifier.fillMaxSize(),
        horizontalAlignment = Alignment.CenterHorizontally,
        verticalArrangement = Arrangement.Center
    ) {
        Text(counter.value.toString(), fontSize = 36.sp)
        Spacer(modifier = Modifier.height(8.dp))
        Button(onClick = onSecondScreen) {
            Text("Go to Second Screen")
        }
    }
}

In [ ]:
@Composable
fun SecondScreen() {
    Column(
        Modifier.fillMaxSize(),
        horizontalAlignment = Alignment.CenterHorizontally,
        verticalArrangement = Arrangement.Center
    ) {
        Text("Second Screen.")
    }
}

In [ ]:
@Composable
fun Navigation() {
    val navController = rememberNavController()
    NavHost(navController = navController, startDestination = Screens.MainScreen.route) {
        composable(route = Screens.MainScreen.route){
            MainScreen{navController.navigate(Screens.SecondScreen.route)}
        }

        composable(route = Screens.SecondScreen.route){
            SecondScreen()
        }
    }
}

In [ ]:
class MainActivity : ComponentActivity() {
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContent {
            CollectAndConvertBasicsComposeTheme {
                Surface(
                    modifier = Modifier.fillMaxSize(),
                    color = MaterialTheme.colorScheme.background
                ) {
                    Navigation()
                }
            }
        }
    }
}

W powyższym kodzie nie ma żadnych nowych elementów.

W rezultacie dostaniemy licznik, którego wartość jest automatycznie inkrementowana. Zwiększanie licznika jest wstrzymane w momencie przejścia na inny ekran (lub w wyniku minimalizacji aplikacji).

Możemy przetestować aplikację

<img src="https://media1.giphy.com/media/v1.Y2lkPTc5MGI3NjExdDZ1N3hvOXM2NXo1cTd3NDM2enlqOWlzOTNsYjJ1M2Uycm52NzlicCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/oVRpNx8YgOGJmka80w/giphy.gif" width="200" />